## 抄袭文本自动判定  
###    ——以新华社新闻数据为例

#### 1.问题转换  
  #### ——定义抄袭文本  
#### 2.数据描述  
#### 3.数据预处理  
#### 4.多模型分类  


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import re

## 一、数据描述

#### 1.数据格式：文本数据，需提取content字段  
#### 2.未有明显标注，需转换问题，来源新华社的数据占比为87%  
#### ——（1）来源于新华社文本设为0，来源于其他设为1，进行0-1分类  
#### ——（2）模型判别来源于新华社，而实际标注为其他，则判定为抄袭文本


In [2]:
PATH=r'E:\NLP\dataset'
content = pd.read_csv(PATH+r'\sqlResult_1558435.csv',encoding='gb18030')

In [3]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [4]:
xinhua_news = content[content.source == '新华社']

In [5]:
len(xinhua_news)/len(content)

0.8778051801676133

In [6]:
news = pd.DataFrame(content)
news['y']=np.where(news.source == '新华社',0,1)
print(len(news[news.y==1])/len(news))
news.head(5)

0.12219481983238666


,id,author,source,content,feature,title,url,y
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,1
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,1
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,1
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,0
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,1


## 二、数据预处理

#### 1.提取content字段新闻数据  
#### 2.分词  
#### 3.tfidf特征提取

In [7]:
#pattern1 = re.compile(r'\n')
pattern1 = re.compile("[\s+\.\!?\/_,$`%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）“”：；【】]+")

In [8]:
def clearing(string,pattern):
    return re.sub(pattern,'',string).replace('\n','')

In [9]:
content_cleaned = [clearing(str(content),pattern1) for content in news.content]

In [10]:
content_cutted = [' '.join(jieba.cut(s)) for s in content_cleaned]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 2.228 seconds.
Prefix dict has been built succesfully.


In [11]:
content_cutted[:2]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 Wi - Fi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台']

In [12]:
tfidf_vectorizer = TfidfVectorizer(encoding='gb18030',use_idf=True)
tfidf_vectors = tfidf_vectorizer.fit_transform(content_cutted)

In [13]:
print(tfidf_vectorizer.get_feature_names()[:20])

['00', '000', '0000', '000001552', '000001631', '00001', '000015', '000019420106', '000019420111001', '000019420222001', '000019420526001', '000019420606001', '000019420921', '000019421123001', '000019422802001', '000019422822001', '000019422828001', '00001hk', '000035', '000038']


In [14]:
print(tfidf_vectors.shape)

(89611, 265011)


In [15]:
print(tfidf_vectors[0])

  (0, 163052)	0.07473691385947241
  (0, 151355)	0.11479395074526405
  (0, 1963)	0.06279795774467459
  (0, 144884)	0.09574354530233725
  (0, 107692)	0.1390335829294175
  (0, 129197)	0.09039849496362332
  (0, 3039)	0.06524648510728423
  (0, 152224)	0.27770608359667676
  (0, 56084)	0.11224607202445959
  (0, 148110)	0.10772334644258012
  (0, 148643)	0.11176970920225676
  (0, 74874)	0.13577286534932317
  (0, 117717)	0.08286902497376143
  (0, 47457)	0.20824240188052168
  (0, 56989)	0.1859643317657902
  (0, 196700)	0.20824240188052168
  (0, 148100)	0.20824240188052168
  (0, 121656)	0.06839508662344358
  (0, 193154)	0.08392544573444331
  (0, 111251)	0.11424532681409844
  (0, 77052)	0.06228168410475248
  (0, 252919)	0.08401569489485615
  (0, 54298)	0.07757855080808931
  (0, 200766)	0.24741720160904213
  (0, 237349)	0.047603922162558876
  (0, 201159)	0.20824240188052168
  (0, 111040)	0.058285887221908785
  (0, 149707)	0.09308881918863887
  (0, 181087)	0.12763851124050393
  (0, 35236)	0.063296287

In [16]:
i = 0
for k,v in tfidf_vectorizer.vocabulary_.items():
    if i < 50:
        print(k,v)
    i+=1

此外 163052
本周 151355
12 1963
日起 144884
小米 107692
手机 129197
15 3039
机型 152224
其余 56084
暂停 148110
更新 148643
发布 74874
开发 117717
体验版 47457
内测 56989
稳定版 196700
暂不受 148100
影响 121656
确保 193154
工程师 111251
可以 77052
集中 252919
全部 54298
精力 200766
进行 237349
系统优化 201159
工作 111040
有人 149707
猜测 181087
主要 35236
用到 185667
miui9 18667
研发 192765
之中 35590
miui8 18666
去年 73385
距今已有 232917
一年 23477
有余 149728
时候 145392
更新换代 148647
当然 121128
关于 55633
确切 193162
信息 50232
我们 127650
还是 236696
等待 199284
官方消息 103557
骁龙 259968


In [17]:
#获取第idx个词向量tfidf得分最高的前n个特征词
def get_feature_names(idx,top_n):
    df = pd.DataFrame(tfidf_vectors[idx].T.todense(), columns=["tfidf"])
    df = df.sort_values(by=["tfidf"],ascending=False)
    feature_idx = df.iloc[:top_n,:].index
    
    features = []
    for k,v in tfidf_vectorizer.vocabulary_.items():
        if v in feature_idx:
            features.append(k)
    
    #print(features)
    #print(df.tfidf[:top_n].values)
    n_features = pd.DataFrame(features,df.tfidf[:top_n].values,columns = ['feature_name'])
    n_features.index.name='tf_idf'
    
    return n_features.reset_index()

get_feature_names(2090,10)

,tf_idf,feature_name
0,0.396362,性能
1,0.300158,测试
2,0.299587,amd
3,0.224463,intel
4,0.208408,锐龙
5,0.204107,corei7
6,0.186124,cpu
7,0.182357,超频
8,0.156306,7820x
9,0.147054,i7


## 三、分类

#### 1.问题定义：  
#### （1）来源于新华社文本设为0，来源于其他设为1，进行0-1分类   
#### （2）模型判别来源于新华社，而实际标注为其他，则判定为抄袭文本
#### 2.多模型分类

In [18]:
import time
def timer(f):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        f(*args, **kwargs)
        end_time = time.time()
        print(end_time - start_time)
    return wrapper

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [20]:
X_train,X_test,y_train,y_test = train_test_split(tfidf_vectors,news.y,test_size = 0.2,random_state=0)

In [21]:
@timer
def predict(str_name,model):
    y_predict = model.predict(X_test)
    target_names = ['source from xinhuashe', 'source from others']
    print(str_name,'：\n', classification_report(y_test,y_predict,target_names=target_names))

### LogisticRegression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
LR = LogisticRegression(random_state=0).fit(X_train,y_train) #baseline
predict('LR',LR)

d:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR ：
                        precision    recall  f1-score   support

source from xinhuashe       0.97      1.00      0.98     15773
   source from others       0.97      0.76      0.85      2150

            micro avg       0.97      0.97      0.97     17923
            macro avg       0.97      0.88      0.92     17923
         weighted avg       0.97      0.97      0.97     17923

0.05191302299499512


In [25]:
LR = LogisticRegression(class_weight = 'balanced', penalty='l1',C = 30 ,solver='liblinear',\
                        random_state=0).fit(X_train,y_train) #特征稀疏设置L1惩罚
predict('LR',LR)

LR ：
                        precision    recall  f1-score   support

source from xinhuashe       1.00      0.99      0.99     15773
   source from others       0.96      0.97      0.96      2150

            micro avg       0.99      0.99      0.99     17923
            macro avg       0.98      0.98      0.98     17923
         weighted avg       0.99      0.99      0.99     17923

0.053861141204833984


### KNN  
class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)[source]

In [37]:
from sklearn.neighbors import KNeighborsClassifier

In [38]:
kNN = KNeighborsClassifier(n_neighbors = 10, weights = 'distance').fit(X_train,y_train) #baseline
predict('KNN',kNN)

KNN ：
                        precision    recall  f1-score   support

source from xinhuashe       0.92      1.00      0.96     15773
   source from others       0.98      0.36      0.53      2150

            micro avg       0.92      0.92      0.92     17923
            macro avg       0.95      0.68      0.74     17923
         weighted avg       0.93      0.92      0.91     17923

206.10255670547485


In [39]:
def weighted(D):
    W = []
    for d in D:
        W.append((D.max() - d) / (D.max() - D.min()))
    return np.array(W)
    

In [40]:
kNN = KNeighborsClassifier(n_neighbors = 3,weights = weighted).fit(X_train,y_train) 
predict('KNN',kNN)

KNN ：
                        precision    recall  f1-score   support

source from xinhuashe       0.94      0.99      0.97     15773
   source from others       0.93      0.51      0.66      2150

            micro avg       0.94      0.94      0.94     17923
            macro avg       0.93      0.75      0.81     17923
         weighted avg       0.94      0.94      0.93     17923

379.65824580192566


### SVM  
class sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)

In [41]:
from sklearn.svm import SVC

In [42]:
svm = SVC(kernel = 'linear', class_weight = 'balanced', random_state = 0).fit(X_train,y_train) #baseline
predict('SVM',svm)

SVM ：
                        precision    recall  f1-score   support

source from xinhuashe       0.99      0.99      0.99     15773
   source from others       0.92      0.95      0.93      2150

            micro avg       0.98      0.98      0.98     17923
            macro avg       0.96      0.97      0.96     17923
         weighted avg       0.98      0.98      0.98     17923

254.4422550201416


In [43]:
svm = SVC(C = 3, kernel = 'rbf', gamma= 0.2, class_weight = 'balanced',random_state = 0).fit(X_train,y_train) #
predict('SVM',svm)

SVM ：
                        precision    recall  f1-score   support

source from xinhuashe       0.99      0.99      0.99     15773
   source from others       0.95      0.95      0.95      2150

            micro avg       0.99      0.99      0.99     17923
            macro avg       0.97      0.97      0.97     17923
         weighted avg       0.99      0.99      0.99     17923

291.563937664032


In [44]:
svm = SVC(C = 3, kernel = 'rbf', gamma= 1.2, class_weight = 'balanced', random_state = 0).fit(X_train,y_train) #
predict('SVM',svm)

SVM ：
                        precision    recall  f1-score   support

source from xinhuashe       0.98      1.00      0.99     15773
   source from others       0.98      0.87      0.92      2150

            micro avg       0.98      0.98      0.98     17923
            macro avg       0.98      0.94      0.96     17923
         weighted avg       0.98      0.98      0.98     17923

7443.992892503738


### Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
#使用全部数据训练发生内存溢出，因此截取部分数据，效果不太好
X_train1,X_test1,y_train1,y_test1 = train_test_split(tfidf_vectors[:3000].toarray(),news.y[:3000],test_size = 0.2,random_state=0) 

In [24]:
bayes = GaussianNB(priors = [0.8,0.2]).fit(X_train1,y_train1)
y_predict = bayes.predict(X_test1)
target_names = ['source from xinhuashe', 'source from others']
print('bayes','：\n', classification_report(y_test1,y_predict,target_names=target_names))

bayes ：
                        precision    recall  f1-score   support

source from xinhuashe       0.78      0.44      0.56        16
   source from others       0.98      1.00      0.99       584

            micro avg       0.98      0.98      0.98       600
            macro avg       0.88      0.72      0.78       600
         weighted avg       0.98      0.98      0.98       600



### Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
dt = DecisionTreeClassifier(criterion = 'gini', splitter = 'best', max_features = 'sqrt', max_depth = 50, \
                            class_weight = 'balanced').fit(X_train,y_train)
predict('Decisoin Tree',dt)

Decisoin Tree ：
                        precision    recall  f1-score   support

source from xinhuashe       0.96      0.56      0.70     15773
   source from others       0.20      0.83      0.33      2150

            micro avg       0.59      0.59      0.59     17923
            macro avg       0.58      0.69      0.52     17923
         weighted avg       0.87      0.59      0.66     17923

0.2693195343017578


In [37]:
dt = DecisionTreeClassifier(criterion = 'gini', splitter = 'best', max_features = 'sqrt', max_depth = 140, min_samples_split = 10,\
                           min_samples_leaf = 2, class_weight = 'balanced',random_state=0 ).fit(X_train,y_train)
predict('Decisoin Tree',dt)

Decisoin Tree ：
                        precision    recall  f1-score   support

source from xinhuashe       0.97      0.93      0.95     15773
   source from others       0.61      0.82      0.70      2150

            micro avg       0.91      0.91      0.91     17923
            macro avg       0.79      0.88      0.82     17923
         weighted avg       0.93      0.91      0.92     17923

0.10870742797851562


### Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rf = RandomForestClassifier(oob_score= True, random_state=1)
rf_clf =rf.fit(X_train,y_train)
print(rf_clf.oob_score_)
predict('随机森林',rf_clf)

d:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
d:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
d:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


0.9424031916080795
随机森林 ：
                        precision    recall  f1-score   support

source from xinhuashe       0.95      1.00      0.97     15773
   source from others       0.98      0.61      0.75      2150

            micro avg       0.95      0.95      0.95     17923
            macro avg       0.96      0.80      0.86     17923
         weighted avg       0.95      0.95      0.95     17923

0.2712736129760742


In [27]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 60, min_samples_split = 10, min_samples_leaf = 2, max_features = 'sqrt',\
                            class_weight = 'balanced', random_state = 1).fit(X_train,y_train)

predict('随机森林',rf)

随机森林 ：
                        precision    recall  f1-score   support

source from xinhuashe       0.99      0.98      0.99     15773
   source from others       0.88      0.95      0.92      2150

            micro avg       0.98      0.98      0.98     17923
            macro avg       0.94      0.97      0.95     17923
         weighted avg       0.98      0.98      0.98     17923

3.1585915088653564
